# Implementing LDA in Python

## Introduction

## Importing the Required Libraries

In [52]:
import pandas as pd
from nltk.corpus import stopwords
import string
import gensim.corpora as corpora
from gensim.models import LdaMulticore

In [5]:
df = pd.read_csv("../data/trc.csv")
df = df[["Last", "First", "Description"]]
df

,Last,First,Description
0,AARON,Thabo Simon,An ANCYL member who was shot and severely inju...
1,ABBOTT,Montaigne,A member of the SADF who was severely injured ...
2,ABRAHAM,Nzaliseko Christopher,A COSAS supporter who was kicked and beaten wi...
3,ABRAHAMS,Achmat Fardiel,Was shot and blinded in one eye by members of ...
4,ABRAHAMS,Annalene Mildred,Was shot and injured by members of the SAP in ...
...,...,...,...
20829,XUZA,Mandla,Was severely injured when he was stoned by a f...
20830,YAKA,Mbangomuni,An IFP supporter and acting induna who was sho...
20831,YALI,Khayalethu,"Was shot by members of the SAP in Lingelihle, ..."
20832,YALO,Bikiwe,An IFP supporter whose house and possessions w...


In [13]:
docs = df.Description.tolist()
docs[:1]

["An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor."]

In [11]:
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [28]:
def clean_doc(doc):
    no_punct = ''
    for c in doc:
        if c not in string.punctuation:
            no_punct = no_punct+c
    # with list comprehension
    # no_punct = ''.join([c for c in doc if c not in string.punctuation])
    
    words = no_punct.lower().split()
    
    final_words = []
    for word in words:
        if word not in stop_words:
            final_words.append(word)
    
    # with list comprehension
    # final_words = [word for word in words if word not in stop_words]

    return final_words
cleaned = clean_doc(docs[0])
print(docs[0])
print(cleaned)

An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor.
['ancyl', 'member', 'shot', 'severely', 'injured', 'sap', 'members', 'lephoi', 'bethulie', 'orange', 'free', 'state', 'ofs', '17', 'april', '1991', 'police', 'opened', 'fire', 'gathering', 'anc', 'supporters', 'house', 'following', 'dispute', 'two', 'neighbours', 'one', 'linked', 'anc', 'sap', 'councillor']


In [31]:
cleaned_docs = [clean_doc(doc) for doc in docs]
print(cleaned_docs[:3])

[['ancyl', 'member', 'shot', 'severely', 'injured', 'sap', 'members', 'lephoi', 'bethulie', 'orange', 'free', 'state', 'ofs', '17', 'april', '1991', 'police', 'opened', 'fire', 'gathering', 'anc', 'supporters', 'house', 'following', 'dispute', 'two', 'neighbours', 'one', 'linked', 'anc', 'sap', 'councillor'], ['member', 'sadf', 'severely', 'injured', 'landmine', 'explosion', 'messina', 'transvaal', '5', 'may', '1987'], ['cosas', 'supporter', 'kicked', 'beaten', 'batons', 'riflebutts', 'members', 'ciskei', 'police', 'protests', 'ciskei', 'government', 'zwelitsha', 'mdantsane', 'ciskei', 'september', '1985']]


## Create ID-Word Index

In [33]:
id2word = corpora.Dictionary(cleaned_docs)

In [37]:
id2word[250]

'bmw'

In [40]:
id_docs = [id2word.doc2bow(cleaned_doc) for cleaned_doc in cleaned_docs]

In [47]:
id_docs[100]

[(17, 1),
 (24, 1),
 (26, 1),
 (30, 1),
 (46, 1),
 (53, 1),
 (55, 1),
 (85, 1),
 (184, 1),
 (621, 1)]

In [46]:
cleaned_docs[100]

['shot',
 'dead',
 'members',
 'sap',
 'protests',
 'cradock',
 'cape',
 '14',
 'august',
 '1987']

In [51]:
for num in id_docs[100]:
    num = num[0]
    print(num, id2word[num])

17 members
24 sap
26 shot
30 1987
46 protests
53 august
55 cape
85 dead
184 14
621 cradock


## Creating LDA Topic Model

In [53]:
topic_nums = 100
lda_model = LdaMulticore(corpus=id_docs, id2word=id2word, num_topics=topic_nums)

In [59]:
lda_model.print_topics(1)

[(79,
  '0.025*"anc" + 0.025*"supporters" + 0.018*"members" + 0.018*"ifp" + 0.015*"attack" + 0.015*"shot" + 0.014*"house" + 0.012*"burnt" + 0.012*"home" + 0.010*"member"')]

## Analyze the Topic Model

In [71]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [75]:
vis = pyLDAvis.gensim_models.prepare(lda_model, id_docs, id2word, mds="mmds", R=30)

In [74]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
74    -0.095960  0.122344       1        1  5.801325
18    -0.076230  0.147627       2        1  4.694428
49    -0.046000  0.114837       3        1  3.846702
25     0.006459 -0.138426       4        1  2.790393
35     0.189159 -0.017354       5        1  2.541542
...         ...       ...     ...      ...       ...
33     0.046857  0.136583      96        1  0.357707
44     0.161011  0.087108      97        1  0.354392
85     0.097405  0.167669      98        1  0.349643
89    -0.032117 -0.089526      99        1  0.334449
71    -0.218634 -0.069188     100        1  0.323972

[100 rows x 5 columns], topic_info=           Term          Freq         Total  Category  logprob  loglift
2           anc  11314.000000  11314.000000   Default  30.0000  30.0000
28   supporters  11018.000000  11018.000000   Default  29.0000  29.0000
347         ifp   8432.000000   8432.000000   Default  28.0000  28.0000
17      members   7125.000000   7125.000000   Default  27.0000  27.0000
49    supporter   8517.000000   8517.000000   Default  26.0000  26.0000
..          ...           ...           ...       ...      ...      ...
246    tortured      8.547402   1123.593705  Topic100  -5.1522   0.8536
13      injured     10.313689   5182.127509  Topic100  -4.9643  -0.4872
158        1993      9.607809   3073.441836  Topic100  -5.0352  -0.0357
55         cape      9.130888   2614.381914  Topic100  -5.0861   0.0751
17      members      9.179574   7125.145217  Topic100  -5.0808  -0.9221

[7455 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
10415     32  0.558125      0025
8434      51  0.615869     03h00
831        2  0.148244       100
831        4  0.029649       100
831        5  0.014824       100
...      ...       ...       ...
9852      79  0.569882    ‘value
8661      66  0.521937   ‘venda’
10463     35  0.424509  ‘wanted’
9047      64  0.435291   “green”
9048      64  0.435265     “red”

[34178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[75, 19, 50, 26, 36, 93, 71, 14, 65, 60, 44, 69, 54, 4, 78, 56, 59, 82, 55, 11, 24, 95, 33, 32, 73, 57, 29, 67, 62, 91, 30, 35, 47, 85, 81, 87, 74, 76, 41, 53, 97, 15, 48, 49, 61, 6, 10, 18, 20, 79, 84, 52, 37, 99, 8, 66, 3, 28, 51, 5, 88, 2, 22, 94, 80, 12, 63, 98, 13, 70, 42, 96, 100, 77, 64, 89, 43, 83, 38, 21, 1, 31, 58, 68, 46, 25, 17, 39, 23, 7, 27, 40, 92, 9, 16, 34, 45, 86, 90, 72])